# Model Training

In [ ]:
import numpy as np
import torch
import os
import cv2

import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset,  Subset
from torchvision import transforms

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using CUDA.")
else:
    device = torch.device("cpu")
    print("Using CPU.")

In [ ]:
# Import data
folder_path = 'video_frames/'

image_paths = os.listdir(folder_path)
image_paths = [folder_path + img_path for img_path in image_paths]

In [ ]:
# Prepare data
y = []
for img in image_paths:
    y.append( cv2.imread(img, cv2.IMREAD_GRAYSCALE) )

y = np.array(y)
X = np.arange(y.shape[0])

X = torch.tensor(X)
#X = nn.functional.one_hot(X, num_classes=y.shape[0]).float()
y = torch.tensor(y).float()
y = y/255 # ensures that values scale between 0 and 1.

#mean = 0.45848998
#std = 0.4895395

#normalize = transforms.Normalize(mean=mean, std=std)
#y = normalize(y)

In [ ]:
num_classes = y.shape[0]
output_dimensions = y[0].shape
print(f"Num classes: {num_classes}")
print(f"Output dimensions: {output_dimensions}")

### Fully connected NN: 

In [ ]:
class Conv_Decoder(nn.Module):
    def __init__(self, latent_dim=256, num_classes=num_classes):
        super().__init__()
        self.latent_dim = latent_dim
        self.num_classes = num_classes

        # FC layer for feature mapping
        self.fc = nn.Linear(num_classes, latent_dim * 3 * 4)

        # Deconvolution layers
        self.deconv_block1 = nn.Sequential(
            nn.ConvTranspose2d(256, 100, 
                               kernel_size=5, stride=5, padding=0),
            nn.BatchNorm2d(100),
            nn.ReLU()
        )
        
        self.deconv_block2 = nn.Sequential(
            nn.ConvTranspose2d(100, 100, 
                               kernel_size=3, stride=3, padding=0),
            nn.BatchNorm2d(100),
            nn.ReLU()
        )
        
        self.deconv_block3 = nn.Sequential(
            nn.ConvTranspose2d(100, 100, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(100),
            nn.ReLU()
        )
        
        self.deconv_block4 = nn.Sequential(
            nn.ConvTranspose2d(100, 100,
                               kernel_size=2, stride=2, padding=0),
            nn.BatchNorm2d(100),
            nn.ReLU()
        )

        self.deconv_block5 = nn.Sequential(
            nn.ConvTranspose2d(100, 1,
                               kernel_size=2, stride=2, padding=0),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # First convolution
        x = nn.functional.one_hot(x, num_classes=self.num_classes).float()
        x = self.fc(x)
        x = x.view(-1, self.latent_dim, 3, 4)
        
        x1 = self.deconv_block1(x)
        x2 = self.deconv_block2(x1)
        x2 = x2 + torch.sum(x)/12  # skip connection
        
        x3 = self.deconv_block3(x2)
        x4 = self.deconv_block4(x3)
        x4 = x4 + torch.sum(x)/12 # skip connection
        
        x5 = self.deconv_block5(x4)
        output = x5.view(-1, 180, 240)
        
        return output

In [ ]:
#model = Conv_Decoder()
model = torch.load('models/Conv_Decoder_Clean.pkl', weights_only=False)
print(model)

In [ ]:
# Hyperparameters
decay_1 = 0.8 # Decay of moving average of gradient
decay_2 = 0.999 # Decay of moving average of squared gradient

lr = 0.00001
weight_decay = 0.0

#lr_decay_rate = 0.99

# Separate bias parameters and non-bias parameters
bias_params = [param for name, param in model.named_parameters() if 'bias' in name]
non_bias_params = [param for name, param in model.named_parameters() if 'bias' not in name]
factor = 10

In [ ]:
# Loss function, Optimizer and Scheduler
criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam([
    {'params': non_bias_params, 'lr': lr}, 
    {'params': bias_params, 'lr': lr * factor}  # increase bias learning rate by factor
], betas=(decay_1, decay_2), weight_decay=weight_decay)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

In [ ]:
model.to(device)
X = X.to(device)
y = y.to(device)

In [ ]:
model_device = next(model.parameters()).device
inputs_device = X.device
labels_device = y.device

print(model_device)
print(inputs_device)
print(labels_device)

In [ ]:
X[2000].shape

In [ ]:
model.forward(torch.tensor(0, dtype=torch.long).to(device))[0].shape

In [ ]:
# Generate randomized sequential indices
seq_length = 6571
start_idx = np.random.randint(0, seq_length)  # Random start
indices = np.arange(start_idx, start_idx + seq_length) % seq_length  # Wrap around

# Create a Subset dataset with randomized sequence start
shuffled_dataset = Subset(TensorDataset(X, y), indices.tolist())

dataloader = DataLoader(shuffled_dataset, batch_size=3, shuffle=False) # both train and test (deliberate overfit) - alternating shuffle

num_epochs = 500
verbose = True

In [ ]:
# Training loop (Repeated many times until convergeance)
loss_values = []
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train() # Set to train mode
    for inputs, labels in dataloader:
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Feed-forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backprop
        loss.backward()
        optimizer.step()
        
        # Add loss
        batch_loss = loss.item()
        running_loss += batch_loss
        loss_values.append(batch_loss)
    
    # Average loss for the epoch
    avg_loss = running_loss / len(dataloader)

    # Update the learning rate at the end of each epoch
    scheduler.step()
    
    if verbose == True:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}")

In [ ]:
# Saves model
torch.save(model, 'models/Conv_Decoder_Clean.pkl')